In [7]:
from pydub import AudioSegment

song =  AudioSegment.from_wav("./test/captcha_6b11d4ebc70e4eb79614c1acc756f1da.wav")

before_value = float('-inf')
non_silent_start = list()
non_silent_end = list()
consecutive_speech = 0

cur_index = 0
for i in song:
    cur_dBFS = i.dBFS
    if cur_dBFS == float("-inf"):
        if before_value == float("-inf"):
            pass
        else:
            non_silent_end.append(cur_index)

    else:
        if before_value == float("-inf"):
            non_silent_start.append(cur_index)
        else:
            consecutive_speech += 1
            
    before_value = cur_dBFS
    if consecutive_speech > 600:
        non_silent_end.append(cur_index)
        before_value = float("-inf")
        consecutive_speech = 0
    
    cur_index += 1

# add last index as the end of non silent. Since there might be sounds that contine till end
non_silent_end.append(len(song))  

In [8]:
non_silent_segment = list()

for i in range(len(non_silent_start)):
    #print("Segment Size : " + str(non_silent_end[i] - non_silent_start[i]))
    if ((non_silent_end[i] - non_silent_start[i]) > 100):
        non_silent_segment.append((non_silent_start[i], non_silent_end[i]))

In [9]:
non_silent_segment

[(108, 511),
 (1416, 1743),
 (2300, 2553),
 (2554, 2885),
 (3514, 3748),
 (3749, 4111),
 (4767, 4985),
 (4986, 5425)]

In [10]:
import os

cur_path = os.getcwd()
out_path = os.path.join(cur_path, "separated")

for i in range(len(non_silent_segment)):
    start = non_silent_segment[i][0]
    end = non_silent_segment[i][1]
    cur_song = song[start:end]
    export_path = os.path.join(out_path, str(i) + ".wav")
    try:
        cur_song.export(export_path, format="wav")
    except FileNotFoundError:  # If directory does not exist
        os.mkdir(out_path)
        cur_song.export(export_path, format="wav")
    print("Exported : " + str(i) + ".wav")

Exported : 0.wav
Exported : 1.wav
Exported : 2.wav
Exported : 3.wav
Exported : 4.wav
Exported : 5.wav
Exported : 6.wav
Exported : 7.wav


In [12]:
from IPython.display import Audio
Audio("./test/captcha_6b11d4ebc70e4eb79614c1acc756f1da.wav")